# Incremental RL

## import packages

In [ ]:
import matplotlib.pyplot as plt

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical
# import gym
import random
# from scipy.interpolate import griddata
from PIL import Image
from collections import deque
import copy
from tqdm import tqdm
from torch.distributions.categorical import Categorical

import os
import math

import pickle

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

## Q learning

In [ ]:
class NoisyLinear(nn.Module):
    """Factorised Gaussian NoisyNet"""

    def __init__(self, in_features, out_features, sigma0=0.25):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.Tensor(out_features, in_features))
        self.bias = nn.Parameter(torch.Tensor(out_features))
        self.noisy_weight = nn.Parameter(
            torch.Tensor(out_features, in_features))
        self.noisy_bias = nn.Parameter(torch.Tensor(out_features))
        self.noise_std = sigma0 / math.sqrt(self.in_features)

        self.reset_parameters()
        self.register_noise()

    def register_noise(self):
        in_noise = torch.FloatTensor(self.in_features)
        out_noise = torch.FloatTensor(self.out_features)
        noise = torch.FloatTensor(self.out_features, self.in_features)
        self.register_buffer('in_noise', in_noise)
        self.register_buffer('out_noise', out_noise)
        self.register_buffer('noise', noise)

    def sample_noise(self):
        self.in_noise.normal_(0, self.noise_std)
        self.out_noise.normal_(0, self.noise_std)
        self.noise = torch.mm(
            self.out_noise.view(-1, 1), self.in_noise.view(1, -1))

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        self.noisy_weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)
            self.noisy_bias.data.uniform_(-stdv, stdv)
            
    def reset_noise_stream(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.noisy_weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.noisy_bias.data.uniform_(-stdv, stdv)

    def forward(self, x):
        """
        Note: noise will be updated if x is not volatile
        """
        normal_y = nn.functional.linear(x, self.weight, self.bias)
        if self.training:
            # update the noise once per update
            self.sample_noise()

        noisy_weight = self.noisy_weight * self.noise
        noisy_bias = self.noisy_bias * self.out_noise
        noisy_y = nn.functional.linear(x, noisy_weight, noisy_bias)
        return noisy_y + normal_y

    def __repr__(self):
        return self.__class__.__name__ + '(' \
            + 'in_features=' + str(self.in_features) \
            + ', out_features=' + str(self.out_features) + ')'

In [ ]:
class DQN(nn.Module):

    def __init__(self, dim=1, window=4, noisy=False):
        # dim: dimension of Expanding World
        # window: the number of timesteps the agent can review
        # noisy: toggle noisy nets
        super().__init__()
        self.dim = dim
        self.window = window
        self.noisy = noisy
        
        # we double the input size for the information of "reward claimed". 
        # 0 for reward claimed (no more reward on that dimension), 1 otherwise
        input_size = dim*2*window 
        output_size = dim*2 + 1    # Additional action for NOOP
        
        # Feedforward policy of 2 hidden layers
        self.policy = nn.Sequential(
            nn.Linear(input_size, 160),
            nn.ReLU(),
            nn.Linear(160, 160),
            nn.ReLU(),
            nn.Linear(160, output_size)
        ) if not noisy else nn.Sequential(
            NoisyLinear(input_size, 160),
            nn.ReLU(),
            NoisyLinear(160, 160),
            nn.ReLU(),
            NoisyLinear(160, output_size)
        )
        
        #\Phi for Relative Frequency
        self.explorer = nn.Sequential(
            nn.Linear(input_size, 40),
            nn.ReLU(),
            nn.Linear(40, output_size)
        )
        #\Psi for epsilon_t
        self.meta_policy = nn.Sequential(
            nn.Linear(input_size, 40),
            nn.ReLU(),
            nn.Linear(40, 2)
        )
        
        #For RND
        self.target = nn.Sequential(
            nn.Linear(input_size, 40),
            nn.ReLU(),
            nn.Linear(40, 40)
        )
        self.predictor = nn.Sequential(
            nn.Linear(input_size, 40),
            nn.ReLU(),
            nn.Linear(40, 40)
        )
        

    def forward(self, state):
        return self.policy(state)
    
    def novelty(self, state):
        return self.explorer(state)
    
    def tradeoff(self, state):
        return self.meta_policy(state)
    
    def intrinsic(self, state):
        t = self.target(state)
        p = self.predictor(state)
        return nn.MSELoss()(t, p)
    
    def add_dim(self):
        self.dim += 1
        
        self.add_input(self.policy, noisy = self.noisy)
        self.add_output(self.policy, noisy = self.noisy)
        
        self.add_input(self.explorer)
        self.add_output(self.explorer, small_bias=True)
        
        self.add_input(self.meta_policy)
#         self.meta_policy = nn.Sequential(
#             nn.Linear(self.dim*2*self.window, 40),
#             nn.ReLU(),
#             nn.Linear(40, 2)
#         )
        
        self.add_input(self.target)
        self.add_input(self.predictor)
    
    def add_input(self, layers, noisy=False):
        '''
        Add an input dimension = 1(dimension)*2(position and remaining rewards)*4(window size) = 8 neurons
        '''
        
        # we're going to deal with the first layer
        index = 0
        
        # take a copy of the current weights stored in self._fc
        old_weight = layers[index].weight.data
        old_bias   = layers[index].bias.data
        device     = old_weight.device
        
        # randomly initialize a tensor with the size of the wanted layer 
        new_weight = torch.zeros([old_weight.shape[0], 2*self.window]).to(device)
        bound = 1 / math.sqrt(old_weight.shape[1]+2*self.window)
        nn.init.uniform_(new_weight, -bound, bound)
        
        
        # concatenate the old weights with the new weights
        new_wi = torch.cat([old_weight, new_weight], dim=1)
        
        # reset weight and grad variables to new size
        if not noisy:
            layers[index] = nn.Linear(old_weight.shape[1]+2*self.window, old_weight.shape[0]).to(device)
        else:
            layers[index] = NoisyLinear(old_weight.shape[1]+2*self.window, old_weight.shape[0]).to(device)

        # set the weight data to new values
        layers[index].weight = torch.nn.Parameter(new_wi).to(device)
        layers[index].bias   = torch.nn.Parameter(old_bias).to(device)
        
    def add_output(self, layers, small_bias=False, noisy=False):
        
        '''
        Add 2 ouput neurons
        '''
        
        # we're going to deal with the last layer
        index = -1
        
        # take a copy of the current weights stored in self._fc
        old_weight = layers[index].weight.data
        old_bias   = layers[index].bias.data
        device     = old_weight.device
        # randomly initialize a tensor with the size of the wanted layer
        
        
        new_weight = torch.zeros([2, old_weight.shape[1]]).to(device)
        bound = 1 / math.sqrt(old_weight.shape[1])
        nn.init.uniform_(new_weight, -bound, bound)
        
        # since we're adding new output neurons, new biases are needed
        new_bias   = torch.zeros(2).to(device)
        if small_bias:
#             constant = old_bias.min()*10 if old_bias.min() < 0 else 0
            nn.init.constant_(new_bias, -100)
        else:
            nn.init.uniform_(new_bias, -bound, bound)
        
        # concatenate the old weights with the new weights
        new_wi = torch.cat([old_weight, new_weight], dim=0)
        new_bi = torch.cat([old_bias, new_bias])

        # reset weight and grad variables to new size
        layers[index] = nn.Linear(old_weight.shape[1], old_weight.shape[0]+2).to(device) if not noisy \
                        else NoisyLinear(old_weight.shape[1], old_weight.shape[0]+2).to(device)

        # set the weight data to new values
        layers[index].weight = torch.nn.Parameter(new_wi).to(device)
        layers[index].bias   = torch.nn.Parameter(new_bi).to(device)

## environment

In [ ]:
class environment():
    def __init__(self, max_dim, cur_dim, window, length, maxSteps):
        self.max_dim = max_dim
        self.cur_dim = cur_dim
        self.window = window
        self.length = length # length of each dimension
        self.maxSteps = maxSteps # max length of an episode
        
        self.reset()
    
        
    def reset(self):
        self.steps = 0
        self.reward = 0
        # second dimension for pos/rewards concatination, third dimension for window concatination
        self.pos = np.zeros((self.cur_dim, 1, 1)) 
        self.rewards = np.ones((self.cur_dim, 1, 1))
        # previous states
        self.history = deque([np.concatenate((np.zeros((cur_dim, 1, 1)), np.ones((cur_dim, 1, 1))), axis=1) for _ in range(window)], maxlen = window)
        
        # return a sequence of states
        return np.concatenate(self.history, axis=2).flatten()
        
    def step(self, action):
        delta = 1/(self.length-1)
        self.steps += 1
        r = 0 # reward
        
        if action == 0:    #NOOP
            r = 0
        else:
            # deduction for the first action (NOOP)
            action = action - 1
            
            # dimension 
            d = action // 2
            
            if action % 2 == 0:
                self.pos[d] = max(0, self.pos[d] - delta)
            else:
                self.pos[d] = min(length, self.pos[d] + delta)
        
        # calculate reward
        # gives reward when the agent is on the very end of any dimension
        # reward is given only when the reward on the dimension is still unclaimed
        for i in range(self.cur_dim):
            if (self.pos[i] == length) and (sum(self.pos) == length and (self.rewards[i] == 1) ):
                self.rewards[i] = 0
                r = 1
                
        
        self.reward += r
        
        #update history
        self.history.append(np.concatenate((self.pos, self.rewards), axis=1))
        
        # check if step limit reached or all rewards claimed
        # if so, return 1 for environment end
        return np.concatenate(self.history, axis=2).flatten(), r, (self.steps >= self.maxSteps or self.reward == self.cur_dim)

## training

In [ ]:
def test(policy, max_dim, cur_dim, length, num_test, epsilon_test, window, max_steps):
    total_reward = 0
    policy.eval()
    
    for i in range(num_test):
        # environment setup
        env = environment(max_dim, cur_dim, window, length, max_steps)
        state = env.reset()
        state = torch.FloatTensor(state)
        
        # run for one episode
        while True:
            with torch.no_grad():
                q = policy(state.to(device))
            if np.random.random() < epsilon_test:
                action = torch.randint(low=0, high=(cur_dim*2)+1, size=(1,))
            else:
                action = q[0:(cur_dim*2)+1].argmax(dim=-1)
            next_state, reward, done = env.step(action)
            next_state = torch.FloatTensor(next_state)
            state = next_state
    
            if done:
                break
                
        total_reward += env.reward
    
    policy.train()
    
    return total_reward / num_test

In [ ]:
# statistics
# ld for linear decay
stats_gamma07 = {'e-greedy':[], 'retrain':[], 'DDE':[], "softmax":[], "argmin":[], "RND":[], "noisy":[], "UCB":[], "DAE":[], "DDE2":[], "e-greedy-ld":[], "DAE-ld":[]}

# load previous stats is available
# with open('stats_gamma07.pkl', 'rb') as f:
#     stats_gamma07 = pickle.load(f)

In [ ]:
num_trials = 20
max_training_steps = 200000 # total training steps
max_steps = 500 # steps per episode
max_dim = 10
cur_dim = 1
length = 2 # length of each dimension
num_test = 1 
test_freq = max_training_steps / 100
gamma = 0.7
epsilon = 0.1
epsilon_test = 0.01
delta = (1-epsilon)/5000. # for linear decay epsilon-greedy
window = 4
lr = 3e-4
loss_fn = nn.MSELoss()

# Q-learning
min_memory = 250
max_memory = 10000
batch_size = 32
target_freq = 300

device = "cuda"

# DAE setting
lr_DAE = 1e-3
td_threshold = 0.5
max_epsilon = 0.2
linear_decay = (1-epsilon) / 5000.

ucb_c   = 1

# DDE for Dynamic Directed epsilon-greedy Exploration: DAE without the meta-policy
# ld for linear decay
for method in ["DAE", 'e-greedy', 'retrain', 'DDE', "argmin", "softmax", "noisy", "UCB", "RND", "DAE-ld", "e-greedy-ld"]:
    for t in range(num_trials):
        if method == "e-greedy-ld" or method == "DAE-ld":
            epsilon = 1
            
        print(f"method: {method}, trial: {t+1}/{num_trials}")
        
        agent = DQN(dim = cur_dim, window = window, noisy = method=="noisy").to(device)
        agent_target = copy.deepcopy(agent).to(device)
        optimizer_policy  = optim.Adam(list(agent.policy.parameters())+list(agent.predictor.parameters()), lr=lr)
        optimizer_DAE = optim.Adam(list(agent.explorer.parameters())+list(agent.meta_policy.parameters()), lr=lr_DAE)
        
        env = environment(max_dim, cur_dim, window, length, max_steps)
        state = env.reset()
        state = torch.FloatTensor(state)

        #init memory
        s_memory   = torch.zeros(max_memory, cur_dim * window * 2)
        a_memory   = torch.zeros(max_memory)
        r_memory   = torch.zeros(max_memory)
        ns_memory  = torch.zeros(max_memory, cur_dim * window * 2)
        num_memory = 0
        memory_idx = 0
        
        ucb_cnt = torch.zeros(cur_dim*2 + 1).to(device) + 1e-5

        # record the number of steps each dimension takes to be well-learned (receiving max rewards in testing)
        learning_curve = [max_training_steps for _ in range(max_dim)]
        # records the relative frequency and epsilon of DAE
        rf_curve = []
        ep_curve_new = [0]
        ep_curve_old = [0]
        
        for step in tqdm(range(max_training_steps)):
            #evaluate the agent
            if (step+1) % test_freq == 0:
                test_reward = test(agent, max_dim, cur_dim, length, num_test, epsilon_test, window, max_steps)
                if test_reward >= cur_dim: # passed the dimension
                    learning_curve[cur_dim-1] = step+1
                    print(f"level {cur_dim} passed !!!!")
                    if cur_dim == max_dim:
                        break
                    else:
                        if method == "e-greedy-ld" or method == "DAE-ld":
                            epsilon = 1
                        cur_dim = cur_dim + 1
                        agent.add_dim()
                        agent = agent.to(device)
                        agent_target.add_dim()
                        agent_target.to(device)
                        
                        if method == "noisy":
                            for module in agent.policy:
                                if isinstance(module, NoisyLinear):
                                    module.reset_noise_stream()
                                    
                        ucb_cnt = torch.cat((ucb_cnt, torch.zeros(2).to(device)+1e-5))
                        optimizer_policy  = optim.Adam(list(agent.policy.parameters())+list(agent.predictor.parameters()), lr=lr)
                        optimizer_DAE = optim.Adam(list(agent.explorer.parameters())+list(agent.meta_policy.parameters()), lr=lr_DAE)
                        env = environment(max_dim, cur_dim, window, length, max_steps)
                        
                        # append zeros to the states in the replay buffer
                        s_memory  = torch.cat([s_memory, torch.zeros(max_memory, window*2)], dim=1)
                        ns_memory = torch.cat([ns_memory, torch.zeros(max_memory, window*2)], dim=1)
                        state = env.reset()
                        state = torch.FloatTensor(state)
                        if method == "retrain":
                            agent = DQN(dim = cur_dim, window = window, noisy = method=="noisy").to(device)
                            agent_target = copy.deepcopy(agent).to(device)
                            optimizer_policy  = optim.Adam(list(agent.policy.parameters())+list(agent.predictor.parameters()), lr=lr)
                            optimizer_DAE    = optim.Adam(list(agent.explorer.parameters())+list(agent.meta_policy.parameters()), lr=lr_DAE)

            #interact
            with torch.no_grad():
                q = agent(state.to(device))

                logit_freq = agent.novelty(state.to(device))
                prob = nn.Softmax(dim=-1)(logit_freq)             
                rf_curve.append(np.concatenate((prob.cpu().numpy(), np.zeros((max_dim-cur_dim)*2)), 0))
                ep = nn.Softmax(dim=-1)(agent.meta_policy(state.to(device)))[1]
                
                # record current epsilon (according to the current state)
                if (step+1)<=20000 or (np.concatenate(env.history, axis=2).flatten()[-8:-4] == 0).all():
                    ep_curve_new.append(ep_curve_new[-1])
                    ep_curve_old.append(ep.cpu().item())
                elif(np.concatenate(env.history, axis=2).flatten()[-8:-4] != 0).all():
                    ep_curve_new.append(ep.cpu().item())
                    ep_curve_old.append(ep_curve_old[-1])
                else:
                    ep_curve_new.append(ep_curve_new[-1])
                    ep_curve_old.append(ep_curve_old[-1])
                
            if method == "softmax":
                action = Categorical(nn.Softmax(-1)(q[0:(cur_dim*2)+1])).sample()
            elif method == "UCB":
                action = (q+ucb_c*torch.sqrt(np.log(step+1)/ucb_cnt))[0:(cur_dim*2)+1].argmax(dim=-1)
            elif method == "DAE":
                if np.random.random() < min(max(epsilon, ep), max_epsilon):
                    action = prob[0:(cur_dim*2)+1].argmin(dim=-1).long()
                    exploration = True
                else:
                    action = q[0:(cur_dim*2)+1].argmax(dim=-1).long()
                    exploration = False
            elif method == "DDE2": # DAE without the explorer
                if np.random.random() < min(max(epsilon, ep), max_epsilon):
                    action = torch.randint(low=0, high=(cur_dim*2)+1, size=(1,)).long()
                    exploration = True
                else:
                    action = q[0:(cur_dim*2)+1].argmax(dim=-1).long()
                    exploration = False
            elif np.random.random() < epsilon:
                if method == "e-greedy" or method == "retrain" or method == "e-greedy-ld":
                    action = torch.randint(low=0, high=(cur_dim*2)+1, size=(1,)).long()
                elif method == "DDE" or method == "DAE-ld":
                    action = prob[0:(cur_dim*2)+1].argmin(dim=-1).long()
                elif method == "argmin":
                    action = q[0:(cur_dim*2)+1].argmin(dim=-1).long()
                else:
                    action = q[0:(cur_dim*2)+1].argmax(dim=-1).long()
            else:
                action = q[0:(cur_dim*2)+1].argmax(dim=-1).long()
            
            #linear decay
            epsilon = max(0.1, epsilon-delta)
            
            ucb_cnt[action] += 1
                
            next_state, reward, done = env.step(action)
            next_state = torch.FloatTensor(next_state)
            
            if method == "RND":
                prediction_error = agent.intrinsic(next_state.to(device))
                reward = reward + prediction_error.item()
                
            #store memory
            s_memory[memory_idx] = state
            a_memory[memory_idx] = action
            r_memory[memory_idx] = reward
            ns_memory[memory_idx] = next_state
            num_memory = min(max_memory, num_memory+1)
            memory_idx = (memory_idx+1)%max_memory

            #learn
            if num_memory >= min_memory:
                idx = random.choices(range(num_memory), k=batch_size)
                s   = s_memory[idx].to(device)
                a   = a_memory[idx].type(torch.long).to(device)
                r   = r_memory[idx].to(device)
                ns  = ns_memory[idx].to(device) 
                
                
                #DAE
                logit_freq = agent.novelty(state.to(device))
                logprob    = nn.LogSoftmax(dim=-1)(logit_freq)
                prob       = nn.Softmax(dim=-1)(logit_freq)
                
                logit_epsilon = agent.tradeoff(state.to(device))
                log_epsilon   = nn.LogSoftmax(dim=-1)(logit_epsilon)
                
                with torch.no_grad():
                    target_value = reward + gamma*agent_target(next_state.to(device))[0:(cur_dim*2)+1].max(dim=-1)[0]
                    predict_value = agent(state.to(device))[action]
                    temp_diff  = nn.L1Loss(reduction='none')(target_value, predict_value)
                    temp_ratio = temp_diff / target_value
                    temp_ratio = temp_ratio.cpu().detach().numpy()
                
                # DQN
                target_values = r + gamma*agent_target(ns)[:,0:(cur_dim*2)+1].max(dim=-1)[0]
                predict_values = agent(s)[range(batch_size), a]
                
                loss = loss_fn(target_values, predict_values)
                
                if method == "DDE" or method == "DAE-ld":
                    loss -= logprob[action]
                if method == "RND":
                    loss += prediction_error
                if method == "DAE":
                    loss -= logprob[action]
                    if exploration:
                        if temp_ratio > td_threshold:
                            loss -= log_epsilon[1]
                        else:
                            loss -= log_epsilon[0]


                optimizer_policy.zero_grad()
                optimizer_DAE.zero_grad()
                loss.backward()
                optimizer_policy.step()
                optimizer_DAE.step()

            if (step+1)  % target_freq == 0:
                agent_target = copy.deepcopy(agent)

            state = next_state
            #check if env done
            if done:
                state = env.reset()
                state = torch.FloatTensor(state)
        
        stats_gamma07[method].append(learning_curve)
        cur_dim = 1

# save stats
with open('stats_gamma07.pkl', 'wb') as f:
    pickle.dump(stats_gamma07, f)

In [ ]:
def adjust_lightness(color, amount=1):
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], max(0, min(1, amount * c[1])), c[2])

In [ ]:
x = [i for i in range(1, max_dim+1)]
plt.figure(figsize=(12, 4.5))

scale = 1000

plt.plot(x, np.array(stats_gamma07["RND"]).mean(0)/scale, label="RND", color=adjust_lightness("tab:red", 1.4), linewidth=5)
plt.plot(x, np.array(stats_gamma07["noisy"]).mean(0)/scale, label="NoisyNet", color=adjust_lightness("tab:orange", 1.4), linewidth=5)
plt.plot(x, np.array(stats_gamma07["softmax"]).mean(0)/scale, label="Softmax", color=adjust_lightness("gold", 1), linewidth=5)
plt.plot(x, np.array(stats_gamma07["UCB"]).mean(0)/scale, label="UCB", color = adjust_lightness("tab:olive", 1), linewidth=5)
plt.plot(x, np.array(stats_gamma07["e-greedy"]).mean(0)/scale, label="ϵ-greedy", color = adjust_lightness("tab:green", 1.6), linewidth=5)
plt.plot(x, np.array(stats_gamma07["argmin"]).mean(0)/scale, label="ArgminQ", color = adjust_lightness("tab:blue", 1.6), linewidth=5)
# plt.plot(x, np.array(stats_gamma07["DDE"]).mean(0), label="DDE(ours)", color="blueviolet", linewidth=5)
plt.plot(x, np.array(stats_gamma07["DAE"]).mean(0)/scale, label='DAE(ours)', color='blueviolet', linewidth=5)
plt.scatter(x, np.array(stats_gamma07["retrain"]).mean(0)/scale, marker='h', label="ϵ-greedy\n(Retrain)", color = "gray", zorder=10, s=128)
plt.xlabel("#dimensions", fontsize=24)
plt.ylabel("training steps (k)", fontsize=24)
plt.xticks(fontsize=0)
plt.yticks(fontsize=24)
axis = plt.gca()
axis.set_ylim([0, 150])
plt.legend(loc='upper left', prop={'size': 17})
plt.tight_layout()
plt.savefig("example.eps", format="eps")
plt.savefig("example.png", format="png")

# Further Analysis of DAE 

In [ ]:
epsilon_curve = {"new_states":[], "old_states":[]}
relative_frequency_curve = []
        
with open('epsilon_curve.pkl', 'rb') as f:
    epsilon_curve = pickle.load(f)
with open('relative_frequency_curve.pkl', 'rb') as f:
    relative_frequency_curve = pickle.load(f)

In [ ]:
num_trials = 20
max_training_steps = 200000 # total training steps
max_steps = 500 # steps per episode
max_dim = 10
cur_dim = 1
length = 2 # length of each dimension
num_test = 1 
test_freq = max_training_steps / 100
gamma = 0.7
epsilon = 0.1
epsilon_test = 0.01
delta = (1-epsilon)/5000. # for linear decay epsilon-greedy
window = 4
lr = 3e-4
loss_fn = nn.MSELoss()

# Q-learning
min_memory = 250
max_memory = 10000
batch_size = 32
target_freq = 300

device = "cuda"

# DAE setting
lr_DAE = 1e-3
td_threshold = 0.5
max_epsilon = 0.2
linear_decay = (1-epsilon) / 5000.

ucb_c   = 1

# DDE for Dynamic Directed epsilon-greedy Exploration: DAE without the meta-policy
# ld for linear decay
for method in ["DAE"]:
    for t in range(num_trials):
        if method == "e-greedy-ld" or method == "DAE-ld":
            epsilon = 1
            
        print(f"method: {method}, trial: {t+1}/{num_trials}")
        
        agent = DQN(dim = cur_dim, window = window, noisy = method=="noisy").to(device)
        agent_target = copy.deepcopy(agent).to(device)
        optimizer_policy  = optim.Adam(list(agent.policy.parameters())+list(agent.predictor.parameters()), lr=lr)
        optimizer_DAE = optim.Adam(list(agent.explorer.parameters())+list(agent.meta_policy.parameters()), lr=lr_DAE)
        
        env = environment(max_dim, cur_dim, window, length, max_steps)
        state = env.reset()
        state = torch.FloatTensor(state)

        #init memory
        s_memory   = torch.zeros(max_memory, cur_dim * window * 2)
        a_memory   = torch.zeros(max_memory)
        r_memory   = torch.zeros(max_memory)
        ns_memory  = torch.zeros(max_memory, cur_dim * window * 2)
        num_memory = 0
        memory_idx = 0
        
        ucb_cnt = torch.zeros(cur_dim*2 + 1).to(device) + 1e-5

        # record the number of steps each dimension takes to be well-learned (receiving max rewards in testing)
        learning_curve = [max_training_steps for _ in range(max_dim)]
        # records the relative frequency and epsilon of DAE
        rf_curve = []
        ep_curve_new = [0]
        ep_curve_old = [0]
        
        for step in tqdm(range(max_training_steps)):
            #evaluate the agent
            if (step+1) % test_freq == 0:
                test_reward = test(agent, max_dim, cur_dim, length, num_test, epsilon_test, window, max_steps)
#                 if test_reward >= cur_dim: # passed the dimension
                if (step+1) % 20000 == 0:
                    learning_curve[cur_dim-1] = step+1
                    if cur_dim == max_dim:
                        break
                    else:
                        if method == "e-greedy-ld" or method == "DAE-ld":
                            epsilon = 1
                        cur_dim = cur_dim + 1
                        agent.add_dim()
                        agent = agent.to(device)
                        agent_target.add_dim()
                        agent_target.to(device)
                        
                        if method == "noisy":
                            for module in agent.policy:
                                if isinstance(module, NoisyLinear):
                                    module.reset_noise_stream()
                                    
                        ucb_cnt = torch.cat((ucb_cnt, torch.zeros(2).to(device)+1e-5))
                        optimizer_policy  = optim.Adam(list(agent.policy.parameters())+list(agent.predictor.parameters()), lr=lr)
                        optimizer_DAE = optim.Adam(list(agent.explorer.parameters())+list(agent.meta_policy.parameters()), lr=lr_DAE)
                        env = environment(max_dim, cur_dim, window, length, max_steps)
                        
                        # append zeros to the states in the replay buffer
                        s_memory  = torch.cat([s_memory, torch.zeros(max_memory, window*2)], dim=1)
                        ns_memory = torch.cat([ns_memory, torch.zeros(max_memory, window*2)], dim=1)
                        state = env.reset()
                        state = torch.FloatTensor(state)
                        if method == "retrain":
                            agent = DQN(dim = cur_dim, window = window, noisy = method=="noisy").to(device)
                            agent_target = copy.deepcopy(agent).to(device)
                            optimizer_policy  = optim.Adam(list(agent.policy.parameters())+list(agent.predictor.parameters()), lr=lr)
                            optimizer_DAE    = optim.Adam(list(agent.explorer.parameters())+list(agent.meta_policy.parameters()), lr=lr_DAE)

            #interact
            with torch.no_grad():
                q = agent(state.to(device))

                logit_freq = agent.novelty(state.to(device))
                prob = nn.Softmax(dim=-1)(logit_freq)             
                rf_curve.append(np.concatenate((prob.cpu().numpy(), np.zeros((max_dim-cur_dim)*2)), 0))
                ep = nn.Softmax(dim=-1)(agent.meta_policy(state.to(device)))[1]
                
                # record current epsilon (according to the current state)
                if (step+1)<=20000 or (np.concatenate(env.history, axis=2).flatten()[-8:-4] == 0).all():
                    ep_curve_new.append(ep_curve_new[-1])
                    ep_curve_old.append(ep.cpu().item())
                elif(np.concatenate(env.history, axis=2).flatten()[-8:-4] != 0).all():
                    ep_curve_new.append(ep.cpu().item())
                    ep_curve_old.append(ep_curve_old[-1])
                else:
                    ep_curve_new.append(ep_curve_new[-1])
                    ep_curve_old.append(ep_curve_old[-1])
                
            if method == "softmax":
                action = Categorical(nn.Softmax(-1)(q[0:(cur_dim*2)+1])).sample()
            elif method == "UCB":
                action = (q+ucb_c*torch.sqrt(np.log(step+1)/ucb_cnt))[0:(cur_dim*2)+1].argmax(dim=-1)
            elif method == "DAE":
                if np.random.random() < min(max(epsilon, ep), max_epsilon):
                    action = prob[0:(cur_dim*2)+1].argmin(dim=-1).long()
                    exploration = True
                else:
                    action = q[0:(cur_dim*2)+1].argmax(dim=-1).long()
                    exploration = False
            elif method == "DDE2": # DAE without the explorer
                if np.random.random() < min(max(epsilon, ep), max_epsilon):
                    action = torch.randint(low=0, high=(cur_dim*2)+1, size=(1,)).long()
                    exploration = True
                else:
                    action = q[0:(cur_dim*2)+1].argmax(dim=-1).long()
                    exploration = False
            elif np.random.random() < epsilon:
                if method == "e-greedy" or method == "retrain" or method == "e-greedy-ld":
                    action = torch.randint(low=0, high=(cur_dim*2)+1, size=(1,)).long()
                elif method == "DDE" or method == "DAE-ld":
                    action = prob[0:(cur_dim*2)+1].argmin(dim=-1).long()
                elif method == "argmin":
                    action = q[0:(cur_dim*2)+1].argmin(dim=-1).long()
                else:
                    action = q[0:(cur_dim*2)+1].argmax(dim=-1).long()
            else:
                action = q[0:(cur_dim*2)+1].argmax(dim=-1).long()
            
            #linear decay
            epsilon = max(0.1, epsilon-delta)
            
            ucb_cnt[action] += 1
                
            next_state, reward, done = env.step(action)
            next_state = torch.FloatTensor(next_state)
            
            if method == "RND":
                prediction_error = agent.intrinsic(next_state.to(device))
                reward = reward + prediction_error.item()
                
            #store memory
            s_memory[memory_idx] = state
            a_memory[memory_idx] = action
            r_memory[memory_idx] = reward
            ns_memory[memory_idx] = next_state
            num_memory = min(max_memory, num_memory+1)
            memory_idx = (memory_idx+1)%max_memory

            #learn
            if num_memory >= min_memory:
                idx = random.choices(range(num_memory), k=batch_size)
                s   = s_memory[idx].to(device)
                a   = a_memory[idx].type(torch.long).to(device)
                r   = r_memory[idx].to(device)
                ns  = ns_memory[idx].to(device) 
                
                
                #DAE
                logit_freq = agent.novelty(state.to(device))
                logprob    = nn.LogSoftmax(dim=-1)(logit_freq)
                prob       = nn.Softmax(dim=-1)(logit_freq)
                
                logit_epsilon = agent.tradeoff(state.to(device))
                log_epsilon   = nn.LogSoftmax(dim=-1)(logit_epsilon)
                
                with torch.no_grad():
                    target_value = reward + gamma*agent_target(next_state.to(device))[0:(cur_dim*2)+1].max(dim=-1)[0]
                    predict_value = agent(state.to(device))[action]
                    temp_diff  = nn.L1Loss(reduction='none')(target_value, predict_value)
                    temp_ratio = temp_diff / target_value
                    temp_ratio = temp_ratio.cpu().detach().numpy()
                
                # DQN
                target_values = r + gamma*agent_target(ns)[:,0:(cur_dim*2)+1].max(dim=-1)[0]
                predict_values = agent(s)[range(batch_size), a]
                
                loss = loss_fn(target_values, predict_values)
                
                if method == "DDE" or method == "DAE-ld":
                    loss -= logprob[action]
                if method == "RND":
                    loss += prediction_error
                if method == "DAE":
                    loss -= logprob[action]
                    if exploration:
                        if temp_ratio > td_threshold:
                            loss -= log_epsilon[1]
                        else:
                            loss -= log_epsilon[0]


                optimizer_policy.zero_grad()
                optimizer_DAE.zero_grad()
                loss.backward()
                optimizer_policy.step()
                optimizer_DAE.step()

            if (step+1)  % target_freq == 0:
                agent_target = copy.deepcopy(agent)

            state = next_state
            #check if env done
            if done:
                state = env.reset()
                state = torch.FloatTensor(state)
        
        relative_frequency_curve.append(rf_curve)
        epsilon_curve["new_states"].append(ep_curve_new[1:])
        epsilon_curve["old_states"].append(ep_curve_old[1:])
        cur_dim = 1

# save stats
with open('epsilon_curve.pkl', 'wb') as f:
    pickle.dump(epsilon_curve, f)
with open('relative_frequency_curve.pkl', 'wb') as f:
    pickle.dump(relative_frequency_curve, f)

# Relative Frequency

In [ ]:
n = len(relative_frequency_curve)
max_len = max([len(c) for c in relative_frequency_curve])
arr = np.ma.empty((n, max_len, 21))
arr.mask = True
for i in range(len(relative_frequency_curve[:10])):
    data = np.array(relative_frequency_curve[i])
    arr[i, :data.shape[0], :data.shape[1]] = data
rf_curve_mean = arr.mean(axis = 0).data

In [ ]:
# divide actions into action groups (A1~A10)
rf_curve_mean = [rf_curve_mean[:, :3].mean(1) if i == 0 else rf_curve_mean[:, 3+(i-1)*2:3+(i)*2].mean(1) for i in range(max_dim)]
rf_curve_mean = np.array(rf_curve_mean)

In [ ]:
moving_average = 10000
moving_averages = []
for i in range(len(rf_curve_mean)):
    ma = np.array([rf_curve_mean[i][:j+1].mean() if j < moving_average else rf_curve_mean[i][j+1-moving_average:j+1].mean() for j in range(len(rf_curve_mean[i]))])
    moving_averages.append(ma)

In [ ]:
x = [i for i in range(len(rf_curve_mean[1]))]
plt.figure(figsize=(12, 5))

pass_time = [i for i in range(20000, 200001, 20000)]
max_prob  = 0

for i, ma in enumerate(moving_averages):
    max_prob = max(ma.max(), max_prob)
    plt.plot(x, ma, label=f'A{i+1}', linewidth=5)
    
for i in range(len(pass_time)-1):
    plt.plot([pass_time[i], pass_time[i]], [0, max_prob], linestyle = 'dashed', color='black')
    

plt.xlabel("training steps", fontsize=24)
plt.ylabel("$\it{Relative Frequency}$", fontsize=24)
axis = plt.gca()
axis.set_xlim([0, 200000])
axis.axes.xaxis.set_ticklabels([])
plt.yticks(fontsize= 24)

plt.legend(loc='upper left', prop={'size': 17})
plt.tight_layout()
plt.savefig("RelativeFrequency.eps", format="eps")
plt.savefig("RelativeFrequency.png", format="png")

# Epsilon

In [ ]:
n = len(epsilon_curve["new_states"])
max_len = 200000
arr = np.ma.empty((n, max_len))
arr.mask = True
for i, curve in enumerate(epsilon_curve["new_states"]):
    arr[i, :len(curve)] = curve
new_mean = arr.mean(axis = 0).data


n = len(epsilon_curve["old_states"])

max_len = 200000
arr = np.ma.empty((n, max_len))
arr.mask = True
for i, curve in enumerate(epsilon_curve["old_states"]):
    arr[i, :len(curve)] = curve
old_mean = arr.mean(axis = 0).data

In [ ]:
x = [i for i in range(len(new_mean))]
plt.figure(figsize=(12, 3))

pass_time = [i for i in range(20000, 200001, 20000)]
max_prob  = 0
moving_average = 1000

ma = np.array([new_mean[:j+1].mean() if j < moving_average else new_mean[j+1-moving_average:j+1].mean() for j in range(len(new_mean))])
max_prob = max(ma.max(), max_prob)
ma[:10000] = 0
plt.plot(x, ma, label='new states', linewidth=5)

ma = np.array([old_mean[:j+1].mean() if j < moving_average else old_mean[j+1-moving_average:j+1].mean() for j in range(len(old_mean))])
max_prob = max(ma.max(), max_prob)
plt.plot(x, ma, label='old states', linewidth=5)
    
for i in range(len(pass_time)-1):
    plt.plot([pass_time[i], pass_time[i]], [0, max_prob], linestyle = 'dashed', color='black')
    

plt.xlabel("training steps", fontsize=24)
plt.ylabel("ϵ$_t$", fontsize=28)
axis = plt.gca()
axis.set_xlim([0, 200000])
axis.axes.xaxis.set_ticklabels([])
axis.axes.get_xaxis().set_visible(False)
plt.yticks(fontsize= 24)

plt.legend(loc='upper left', prop={'size': 24})
plt.tight_layout()
plt.savefig("epsilon_curve.eps", format="eps")
plt.savefig("epsilon_curve.png", format="png")